# Day 7: Coffee Automaton - Interactive Complexity Dynamics Exploration

*"Understanding how complexity emerges from simple rules through the lens of a cooling coffee cup"*

This interactive notebook explores complexity theory through the Coffee Automaton model, demonstrating how simple local rules can generate rich, emergent behaviors that mirror fundamental principles in AI and physics.

## Learning Objectives
- Understand complexity emergence from simple rules
- Explore entropy dynamics in computational systems  
- Analyze phase transitions and critical phenomena
- Connect complexity theory to modern AI architectures
- Visualize the "sweet spot" where complexity thrives

Let's begin our journey into the fascinating world of complexity dynamics!

In [ ]:
# Import all necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings
warnings.filterwarnings('ignore')

# Import our custom implementations
import sys
sys.path.append('.')
from implementation import CoffeeAutomaton, ComplexityMeasures, LifeAnalyzer
from visualization import CoffeeAutomatonVisualizer

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("🔬 Coffee Automaton Interactive Lab Ready!")
print("📊 All libraries loaded successfully")

## Part 1: Understanding the Coffee Automaton

The Coffee Automaton models how a coffee cup cools down through cellular automata rules. Each cell represents a small region of coffee, and the temperature evolves according to:

**Heat Diffusion Rule**: 
```
T_new[i,j] = T_old[i,j] + α × (ΣT_neighbors - 4×T_old[i,j]) + noise
```

**Environmental Cooling**: 
```
T_new[i,j] = T_new[i,j] × (1 - cooling_rate)
```

Let's create our first coffee automaton and watch it evolve!

In [ ]:
# Create a basic coffee automaton
def create_coffee_demo(size=50, initial_temp=100.0):
    """Create and display a basic coffee automaton."""
    
    # Initialize the automaton
    coffee = CoffeeAutomaton(size=size, initial_temp=initial_temp)
    
    print(f"☕ Created {size}×{size} coffee automaton")
    print(f"🌡️ Initial temperature: {initial_temp}°C")
    print(f"❄️ Environment temperature: {coffee.environment_temp}°C")
    
    # Run a few steps and show the evolution
    temps = []
    complexities = []
    entropies = []
    
    for step in range(20):
        temps.append(coffee.grid.mean())
        
        # Calculate complexity metrics
        complexity_calc = ComplexityMeasures()
        complexity = complexity_calc.calculate_local_complexity(coffee.grid)
        entropy = complexity_calc.calculate_entropy(coffee.grid)
        
        complexities.append(complexity.mean())
        entropies.append(entropy)
        
        coffee.step()
    
    # Plot the evolution
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))
    
    # Temperature evolution
    ax1.plot(temps, 'r-o', linewidth=2, markersize=6)
    ax1.set_xlabel('Time Steps')
    ax1.set_ylabel('Average Temperature (°C)')
    ax1.set_title('Coffee Cooling Dynamics')
    ax1.grid(True, alpha=0.3)
    
    # Complexity evolution  
    ax2.plot(complexities, 'b-s', linewidth=2, markersize=6)
    ax2.set_xlabel('Time Steps')
    ax2.set_ylabel('Local Complexity')
    ax2.set_title('Complexity Emergence')
    ax2.grid(True, alpha=0.3)
    
    # Entropy evolution
    ax3.plot(entropies, 'g-^', linewidth=2, markersize=6)
    ax3.set_xlabel('Time Steps')
    ax3.set_ylabel('System Entropy')
    ax3.set_title('Information Dynamics')
    ax3.grid(True, alpha=0.3)
    
    # Final temperature distribution
    im = ax4.imshow(coffee.grid, cmap='hot', interpolation='bilinear')
    ax4.set_title('Final Temperature Distribution')
    plt.colorbar(im, ax=ax4, label='Temperature (°C)')
    
    plt.tight_layout()
    plt.show()
    
    return coffee

# Create and run demo
demo_coffee = create_coffee_demo(size=40, initial_temp=95.0)

## Part 2: Interactive Parameter Exploration

Now let's create interactive widgets to explore how different parameters affect the coffee automaton's behavior. This is where the magic happens - we'll see how small changes can lead to dramatically different complexity patterns!

In [ ]:
# Interactive parameter exploration
def explore_parameters(diffusion_rate=0.1, cooling_rate=0.02, noise_level=0.01, size=30):
    """Interactive exploration of coffee automaton parameters."""
    
    # Create automaton with custom parameters
    coffee = CoffeeAutomaton(
        size=size, 
        initial_temp=100.0,
        diffusion_rate=diffusion_rate,
        cooling_rate=cooling_rate,
        noise_level=noise_level
    )
    
    # Run simulation
    steps = 30
    metrics = {'temp': [], 'complexity': [], 'entropy': []}
    grids = []
    
    complexity_calc = ComplexityMeasures()
    
    for step in range(steps):
        # Store metrics
        metrics['temp'].append(coffee.grid.mean())
        
        complexity = complexity_calc.calculate_local_complexity(coffee.grid)
        metrics['complexity'].append(complexity.mean())
        
        entropy = complexity_calc.calculate_entropy(coffee.grid)
        metrics['entropy'].append(entropy)
        
        # Store grid snapshots
        if step % 10 == 0:
            grids.append(coffee.grid.copy())
        
        coffee.step()
    
    # Create visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
    
    # Metrics evolution
    time_steps = range(steps)
    ax1.plot(time_steps, metrics['temp'], 'r-', linewidth=2, label='Temperature')
    ax1_twin = ax1.twinx()
    ax1_twin.plot(time_steps, metrics['complexity'], 'b-', linewidth=2, label='Complexity')
    ax1_twin.plot(time_steps, metrics['entropy'], 'g-', linewidth=2, label='Entropy')
    
    ax1.set_xlabel('Time Steps')
    ax1.set_ylabel('Temperature (°C)', color='red')
    ax1_twin.set_ylabel('Complexity / Entropy', color='blue')
    ax1.set_title(f'System Evolution (diff={diffusion_rate:.3f}, cool={cooling_rate:.3f})')
    
    # Temperature distributions at different times
    for i, (grid, step) in enumerate(zip(grids, [0, 10, 20])):
        ax = [ax2, ax3, ax4][i]
        im = ax.imshow(grid, cmap='hot', interpolation='bilinear', vmin=0, vmax=100)
        ax.set_title(f'Step {step}')
        ax.set_xticks([])
        ax.set_yticks([])
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    plt.tight_layout()
    plt.show()
    
    # Print insights
    max_complexity = max(metrics['complexity'])
    max_complexity_step = metrics['complexity'].index(max_complexity)
    
    print(f"📊 Parameter Analysis:")
    print(f"   Diffusion Rate: {diffusion_rate:.3f}")
    print(f"   Cooling Rate: {cooling_rate:.3f}")
    print(f"   Noise Level: {noise_level:.3f}")
    print(f"   Peak Complexity: {max_complexity:.3f} at step {max_complexity_step}")
    print(f"   Final Temperature: {metrics['temp'][-1]:.2f}°C")

# Create interactive widget
interact(explore_parameters,
         diffusion_rate=widgets.FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, description='Diffusion:'),
         cooling_rate=widgets.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.02, description='Cooling:'),
         noise_level=widgets.FloatSlider(min=0.0, max=0.05, step=0.001, value=0.01, description='Noise:'),
         size=widgets.IntSlider(min=20, max=60, step=10, value=30, description='Grid Size:'))

## Part 3: The Complexity Sweet Spot

One of the most fascinating discoveries in complexity science is the existence of a "sweet spot" - parameter regimes where complexity is maximized. This occurs at the edge of chaos, between order and disorder.

Let's systematically explore this sweet spot in our coffee automaton!

In [ ]:
# Systematic exploration of the complexity landscape
def find_complexity_landscape():
    """Map the complexity landscape across parameter space."""
    
    print("🔍 Mapping complexity landscape... (this may take a moment)")
    
    # Parameter ranges to explore
    diffusion_rates = np.linspace(0.05, 0.3, 8)
    cooling_rates = np.linspace(0.01, 0.08, 8)
    
    complexity_map = np.zeros((len(cooling_rates), len(diffusion_rates)))
    entropy_map = np.zeros((len(cooling_rates), len(diffusion_rates)))
    
    complexity_calc = ComplexityMeasures()
    
    for i, cooling in enumerate(cooling_rates):
        for j, diffusion in enumerate(diffusion_rates):
            # Create automaton with these parameters
            coffee = CoffeeAutomaton(
                size=25,  # Smaller for speed
                diffusion_rate=diffusion,
                cooling_rate=cooling,
                noise_level=0.01
            )
            
            # Run simulation and collect metrics
            complexities = []
            entropies = []
            
            for step in range(25):
                complexity = complexity_calc.calculate_local_complexity(coffee.grid)
                entropy = complexity_calc.calculate_entropy(coffee.grid)
                
                complexities.append(complexity.mean())
                entropies.append(entropy)
                
                coffee.step()
            
            # Store peak complexity and entropy
            complexity_map[i, j] = max(complexities)
            entropy_map[i, j] = max(entropies)
    
    # Visualize the landscape
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Complexity landscape
    im1 = ax1.imshow(complexity_map, cmap='viridis', aspect='auto', origin='lower')
    ax1.set_xticks(range(len(diffusion_rates)))
    ax1.set_yticks(range(len(cooling_rates)))
    ax1.set_xticklabels([f'{d:.2f}' for d in diffusion_rates])
    ax1.set_yticklabels([f'{c:.3f}' for c in cooling_rates])
    ax1.set_xlabel('Diffusion Rate')
    ax1.set_ylabel('Cooling Rate')
    ax1.set_title('Complexity Landscape')
    plt.colorbar(im1, ax=ax1, label='Peak Complexity')
    
    # Entropy landscape
    im2 = ax2.imshow(entropy_map, cmap='plasma', aspect='auto', origin='lower')
    ax2.set_xticks(range(len(diffusion_rates)))
    ax2.set_yticks(range(len(cooling_rates)))
    ax2.set_xticklabels([f'{d:.2f}' for d in diffusion_rates])
    ax2.set_yticklabels([f'{c:.3f}' for c in cooling_rates])
    ax2.set_xlabel('Diffusion Rate')
    ax2.set_ylabel('Cooling Rate')
    ax2.set_title('Entropy Landscape')
    plt.colorbar(im2, ax=ax2, label='Peak Entropy')
    
    plt.tight_layout()
    plt.show()
    
    # Find the sweet spots
    max_complexity_idx = np.unravel_index(complexity_map.argmax(), complexity_map.shape)
    max_entropy_idx = np.unravel_index(entropy_map.argmax(), entropy_map.shape)
    
    print(f"🎯 Sweet Spot Analysis:")
    print(f"   Maximum Complexity: {complexity_map.max():.3f}")
    print(f"     at diffusion={diffusion_rates[max_complexity_idx[1]]:.3f}, cooling={cooling_rates[max_complexity_idx[0]]:.3f}")
    print(f"   Maximum Entropy: {entropy_map.max():.3f}")
    print(f"     at diffusion={diffusion_rates[max_entropy_idx[1]]:.3f}, cooling={cooling_rates[max_entropy_idx[0]]:.3f}")
    
    return diffusion_rates[max_complexity_idx[1]], cooling_rates[max_complexity_idx[0]]

# Find the sweet spot
optimal_diffusion, optimal_cooling = find_complexity_landscape()

## Part 4: Life in the Coffee Cup

Now let's explore the fascinating emergence of "life-like" patterns in our coffee automaton. These patterns exhibit characteristics similar to Conway's Game of Life but emerge from physical heat diffusion rules.

In [ ]:
# Explore life-like patterns in optimal conditions
def explore_coffee_life(diffusion_rate, cooling_rate):
    """Explore life-like patterns in the coffee automaton."""
    
    print(f"🔬 Exploring life patterns with optimal parameters:")
    print(f"   Diffusion: {diffusion_rate:.3f}, Cooling: {cooling_rate:.3f}")
    
    # Create automaton with optimal parameters
    coffee = CoffeeAutomaton(
        size=40,
        diffusion_rate=diffusion_rate,
        cooling_rate=cooling_rate,
        noise_level=0.015  # Slightly higher noise for pattern formation
    )
    
    # Add some "seeds" for pattern formation
    coffee.add_hotspot(15, 15, intensity=20, radius=3)
    coffee.add_hotspot(25, 25, intensity=25, radius=2)
    coffee.add_hotspot(20, 30, intensity=15, radius=4)
    
    life_analyzer = LifeAnalyzer()
    
    # Run simulation and analyze patterns
    grids = []
    life_metrics = []
    
    for step in range(50):
        # Store grid
        if step % 5 == 0:
            grids.append(coffee.grid.copy())
        
        # Analyze life-like behavior
        metrics = life_analyzer.analyze_step(coffee.grid)
        life_metrics.append(metrics)
        
        coffee.step()
    
    # Visualize the evolution
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    
    # Show grid evolution
    for i, grid in enumerate(grids):
        ax = axes[0, i]
        im = ax.imshow(grid, cmap='hot', interpolation='nearest', vmin=0, vmax=100)
        ax.set_title(f'Step {i*5}')
        ax.set_xticks([])
        ax.set_yticks([])
        if i == 4:
            plt.colorbar(im, ax=ax)
    
    # Show life pattern analysis
    for i, grid in enumerate(grids):
        ax = axes[1, i]
        # Threshold to show "alive" cells
        alive_pattern = (grid > coffee.environment_temp + 10).astype(float)
        im = ax.imshow(alive_pattern, cmap='RdYlBu_r', interpolation='nearest')
        ax.set_title(f'Life Pattern {i*5}')
        ax.set_xticks([])
        ax.set_yticks([])
        if i == 4:
            plt.colorbar(im, ax=ax)
    
    plt.tight_layout()
    plt.show()
    
    # Plot life metrics
    if life_metrics:
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
        
        steps = range(len(life_metrics))
        
        # Extract metrics
        oscillator_counts = [m.get('oscillator_count', 0) for m in life_metrics]
        still_life_counts = [m.get('still_life_count', 0) for m in life_metrics]
        pattern_diversity = [m.get('pattern_diversity', 0) for m in life_metrics]
        active_regions = [m.get('active_regions', 0) for m in life_metrics]
        
        ax1.plot(steps, oscillator_counts, 'b-o', linewidth=2, label='Oscillators')
        ax1.set_xlabel('Time Steps')
        ax1.set_ylabel('Count')
        ax1.set_title('Oscillating Patterns')
        ax1.grid(True, alpha=0.3)
        
        ax2.plot(steps, still_life_counts, 'r-s', linewidth=2, label='Still Life')
        ax2.set_xlabel('Time Steps')
        ax2.set_ylabel('Count')
        ax2.set_title('Stable Patterns')
        ax2.grid(True, alpha=0.3)
        
        ax3.plot(steps, pattern_diversity, 'g-^', linewidth=2, label='Diversity')
        ax3.set_xlabel('Time Steps')
        ax3.set_ylabel('Diversity Index')
        ax3.set_title('Pattern Diversity')
        ax3.grid(True, alpha=0.3)
        
        ax4.plot(steps, active_regions, 'm-d', linewidth=2, label='Active Regions')
        ax4.set_xlabel('Time Steps')
        ax4.set_ylabel('Count')
        ax4.set_title('Active Regions')
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print(f"📈 Life Pattern Analysis:")
        print(f"   Peak Oscillators: {max(oscillator_counts) if oscillator_counts else 0}")
        print(f"   Stable Patterns: {max(still_life_counts) if still_life_counts else 0}")
        print(f"   Pattern Diversity: {max(pattern_diversity) if pattern_diversity else 0:.3f}")

# Explore life with optimal parameters
explore_coffee_life(optimal_diffusion, optimal_cooling)

## Part 5: Connections to AI and Modern Systems

The Coffee Automaton reveals fundamental principles that appear throughout AI and complex systems. Let's explore these connections and understand why complexity theory matters for modern machine learning.

In [ ]:
# Connections to neural networks and AI
def demonstrate_ai_connections():
    """Show connections between coffee automaton and AI systems."""
    
    print("🧠 Exploring connections to AI and neural networks...")
    
    # Create a coffee automaton that mimics neural network dynamics
    class NeuralCoffeeAutomaton(CoffeeAutomaton):
        """Coffee automaton with neural-network-like dynamics."""
        
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # Add "synaptic weights" between cells
            self.weights = np.random.randn(self.size, self.size, 8) * 0.1
            
        def step(self):
            # Standard coffee dynamics
            super().step()
            
            # Add neural-like activation
            activation = np.tanh((self.grid - self.environment_temp) / 10.0)
            self.grid = self.environment_temp + activation * 30
    
    # Compare standard vs neural coffee
    coffee_standard = CoffeeAutomaton(size=30, diffusion_rate=0.15, cooling_rate=0.03)
    coffee_neural = NeuralCoffeeAutomaton(size=30, diffusion_rate=0.15, cooling_rate=0.03)
    
    # Add identical initial conditions
    coffee_standard.add_hotspot(15, 15, intensity=30, radius=5)
    coffee_neural.add_hotspot(15, 15, intensity=30, radius=5)
    
    # Run both simulations
    standard_grids = []
    neural_grids = []
    
    for step in range(30):
        if step % 6 == 0:
            standard_grids.append(coffee_standard.grid.copy())
            neural_grids.append(coffee_neural.grid.copy())
        
        coffee_standard.step()
        coffee_neural.step()
    
    # Visualize comparison
    fig, axes = plt.subplots(2, 5, figsize=(18, 8))
    
    for i, (std_grid, neural_grid) in enumerate(zip(standard_grids, neural_grids)):
        # Standard coffee
        im1 = axes[0, i].imshow(std_grid, cmap='hot', vmin=20, vmax=80)
        axes[0, i].set_title(f'Standard Step {i*6}')
        axes[0, i].set_xticks([])
        axes[0, i].set_yticks([])
        
        # Neural coffee
        im2 = axes[1, i].imshow(neural_grid, cmap='hot', vmin=20, vmax=80)
        axes[1, i].set_title(f'Neural Step {i*6}')
        axes[1, i].set_xticks([])
        axes[1, i].set_yticks([])
    
    axes[0, 0].set_ylabel('Standard Coffee', rotation=90, size=12)
    axes[1, 0].set_ylabel('Neural Coffee', rotation=90, size=12)
    
    plt.tight_layout()
    plt.show()
    
    # Analyze differences
    complexity_calc = ComplexityMeasures()
    
    std_complexity = complexity_calc.calculate_local_complexity(standard_grids[-1]).mean()
    neural_complexity = complexity_calc.calculate_local_complexity(neural_grids[-1]).mean()
    
    std_entropy = complexity_calc.calculate_entropy(standard_grids[-1])
    neural_entropy = complexity_calc.calculate_entropy(neural_grids[-1])
    
    print(f"📊 Complexity Comparison:")
    print(f"   Standard Coffee - Complexity: {std_complexity:.3f}, Entropy: {std_entropy:.3f}")
    print(f"   Neural Coffee   - Complexity: {neural_complexity:.3f}, Entropy: {neural_entropy:.3f}")
    
    # Show AI principles demonstrated
    print(f"\n🎯 AI Principles Demonstrated:")
    print(f"   🔄 Emergent Behavior: Complex patterns from simple rules")
    print(f"   📊 Information Processing: Entropy and complexity dynamics")
    print(f"   🎚️ Critical Dynamics: Edge-of-chaos optimal performance")
    print(f"   🔗 Network Effects: Local interactions → Global patterns")
    print(f"   🧮 Nonlinear Dynamics: Small changes → Large effects")

demonstrate_ai_connections()

## Part 6: Advanced Analysis and Experiments

Let's dive deeper into advanced aspects of the coffee automaton, including phase transitions, criticality, and information flow dynamics.

In [ ]:
# Advanced analysis: Phase transitions and criticality
def analyze_phase_transitions():
    """Analyze phase transitions in the coffee automaton."""
    
    print("🌡️ Analyzing phase transitions and critical phenomena...")
    
    # Explore different noise levels to find phase transitions
    noise_levels = np.logspace(-3, -1, 20)  # From 0.001 to 0.1
    
    order_parameters = []
    susceptibilities = []
    
    for noise in noise_levels:
        # Run multiple realizations
        order_vals = []
        
        for realization in range(5):
            coffee = CoffeeAutomaton(
                size=25,
                diffusion_rate=0.12,
                cooling_rate=0.025,
                noise_level=noise
            )
            
            # Add initial perturbation
            coffee.add_hotspot(12, 12, intensity=20, radius=4)
            
            # Let system evolve
            for step in range(40):
                coffee.step()
            
            # Calculate order parameter (temperature variance)
            order = np.var(coffee.grid)
            order_vals.append(order)
        
        # Store statistics
        mean_order = np.mean(order_vals)
        susceptibility = np.var(order_vals)  # Response to noise
        
        order_parameters.append(mean_order)
        susceptibilities.append(susceptibility)
    
    # Plot phase transition analysis
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Order parameter
    ax1.semilogx(noise_levels, order_parameters, 'bo-', linewidth=2, markersize=6)
    ax1.set_xlabel('Noise Level')
    ax1.set_ylabel('Order Parameter (Temperature Variance)')
    ax1.set_title('Order-Disorder Transition')
    ax1.grid(True, alpha=0.3)
    
    # Susceptibility (critical behavior)
    ax2.semilogx(noise_levels, susceptibilities, 'rs-', linewidth=2, markersize=6)
    ax2.set_xlabel('Noise Level')
    ax2.set_ylabel('Susceptibility')
    ax2.set_title('Critical Behavior')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Find critical point
    max_susceptibility_idx = np.argmax(susceptibilities)
    critical_noise = noise_levels[max_susceptibility_idx]
    
    print(f"🎯 Critical Point Analysis:")
    print(f"   Critical noise level: {critical_noise:.4f}")
    print(f"   Maximum susceptibility: {max(susceptibilities):.3f}")
    print(f"   This represents the edge-of-chaos transition!")

analyze_phase_transitions()

## Part 7: Your Turn to Explore!

Now it's your turn to experiment with the Coffee Automaton! Try different parameter combinations, initial conditions, or even modify the rules to see what happens.

### Suggested Experiments:

1. **Boundary Effects**: Try different boundary conditions (periodic vs fixed)
2. **Multiple Hotspots**: Add multiple heat sources and watch them interact
3. **Custom Rules**: Modify the diffusion or cooling rules
4. **Size Effects**: Explore how system size affects complexity
5. **Time Scales**: Investigate very long-term behavior

Use the cells below to conduct your own experiments!

In [ ]:
# Experiment cell 1: Your custom experiment here
def my_experiment():
    """Design your own coffee automaton experiment!"""
    
    print("🔬 Your Custom Experiment")
    
    # TODO: Design your experiment here!
    # Ideas:
    # - Try extreme parameter values
    # - Create custom initial conditions
    # - Implement new rules
    # - Analyze long-term behavior
    
    # Example: Multiple interacting hotspots
    coffee = CoffeeAutomaton(size=40, diffusion_rate=0.08, cooling_rate=0.02, noise_level=0.005)
    
    # Add multiple hotspots in a pattern
    hotspot_positions = [(10, 10), (30, 30), (10, 30), (30, 10), (20, 20)]
    intensities = [25, 20, 15, 30, 35]
    
    for (x, y), intensity in zip(hotspot_positions, intensities):
        coffee.add_hotspot(x, y, intensity=intensity, radius=3)
    
    # Run and visualize
    grids = []
    for step in range(60):
        if step % 10 == 0:
            grids.append(coffee.grid.copy())
        coffee.step()
    
    # Show evolution
    fig, axes = plt.subplots(1, len(grids), figsize=(18, 3))
    for i, grid in enumerate(grids):
        axes[i].imshow(grid, cmap='hot', vmin=20, vmax=60)
        axes[i].set_title(f'Step {i*10}')
        axes[i].set_xticks([])
        axes[i].set_yticks([])
    
    plt.tight_layout()
    plt.show()
    
    print("🎉 Experiment complete! What patterns do you observe?")

# Run your experiment
my_experiment()

In [ ]:
# Experiment cell 2: Advanced analysis
def advanced_analysis_experiment():
    """Perform advanced analysis on your coffee automaton."""
    
    print("📈 Advanced Analysis Experiment")
    
    # Create a coffee automaton with interesting dynamics
    coffee = CoffeeAutomaton(size=50, diffusion_rate=0.1, cooling_rate=0.03, noise_level=0.008)
    
    # Add a spiral pattern as initial condition
    center_x, center_y = 25, 25
    for angle in np.linspace(0, 4*np.pi, 100):
        x = int(center_x + 10 * np.cos(angle) * (1 - angle/(4*np.pi)))
        y = int(center_y + 10 * np.sin(angle) * (1 - angle/(4*np.pi)))
        if 0 <= x < coffee.size and 0 <= y < coffee.size:
            coffee.grid[x, y] = 80
    
    # Run simulation with detailed tracking
    complexity_calc = ComplexityMeasures()
    
    metrics_history = {
        'complexity': [],
        'entropy': [],
        'temperature': [],
        'spatial_correlation': []
    }
    
    for step in range(100):
        # Calculate various metrics
        complexity = complexity_calc.calculate_local_complexity(coffee.grid)
        entropy = complexity_calc.calculate_entropy(coffee.grid)
        
        # Spatial correlation (how correlated are neighboring cells?)
        shifted = np.roll(coffee.grid, 1, axis=0)
        correlation = np.corrcoef(coffee.grid.flatten(), shifted.flatten())[0,1]
        
        metrics_history['complexity'].append(complexity.mean())
        metrics_history['entropy'].append(entropy)
        metrics_history['temperature'].append(coffee.grid.mean())
        metrics_history['spatial_correlation'].append(correlation)
        
        coffee.step()
    
    # Plot comprehensive analysis
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))
    
    time_steps = range(len(metrics_history['complexity']))
    
    ax1.plot(time_steps, metrics_history['complexity'], 'b-', linewidth=2, label='Complexity')
    ax1.set_xlabel('Time Steps')
    ax1.set_ylabel('Local Complexity')
    ax1.set_title('Complexity Evolution')
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(time_steps, metrics_history['entropy'], 'r-', linewidth=2, label='Entropy')
    ax2.set_xlabel('Time Steps')
    ax2.set_ylabel('System Entropy')
    ax2.set_title('Information Dynamics')
    ax2.grid(True, alpha=0.3)
    
    ax3.plot(time_steps, metrics_history['temperature'], 'orange', linewidth=2, label='Temperature')
    ax3.set_xlabel('Time Steps')
    ax3.set_ylabel('Average Temperature')
    ax3.set_title('Thermal Evolution')
    ax3.grid(True, alpha=0.3)
    
    ax4.plot(time_steps, metrics_history['spatial_correlation'], 'purple', linewidth=2, label='Correlation')
    ax4.set_xlabel('Time Steps')
    ax4.set_ylabel('Spatial Correlation')
    ax4.set_title('Spatial Structure')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Analysis Results:")
    print(f"   Peak complexity: {max(metrics_history['complexity']):.3f}")
    print(f"   Final entropy: {metrics_history['entropy'][-1]:.3f}")
    print(f"   Temperature drop: {metrics_history['temperature'][0] - metrics_history['temperature'][-1]:.2f}°C")
    print(f"   Final correlation: {metrics_history['spatial_correlation'][-1]:.3f}")

# Run advanced analysis
advanced_analysis_experiment()

## Conclusions and Takeaways

🎉 **Congratulations!** You've completed an in-depth exploration of complexity dynamics through the Coffee Automaton model.

### Key Insights Discovered:

1. **Emergence**: Complex behaviors arise from simple local rules
2. **Sweet Spots**: Optimal complexity occurs at specific parameter regimes
3. **Phase Transitions**: Systems exhibit critical behavior at transition points
4. **Information Flow**: Entropy and complexity provide insights into system dynamics
5. **Life-like Patterns**: Self-organizing structures emerge naturally
6. **AI Connections**: Same principles apply to neural networks and machine learning

### Why This Matters for AI:

- **Neural Network Design**: Understanding how complexity emerges helps design better architectures
- **Optimization Landscapes**: Critical dynamics inform training strategies
- **Emergent Behavior**: Insight into how AI systems develop unexpected capabilities
- **Information Processing**: Entropy principles guide efficient computation
- **Self-Organization**: Understanding how structure emerges without explicit programming

### Next Steps:

1. **Experiment Further**: Try your own parameter combinations and initial conditions
2. **Connect to Research**: Read papers on complexity theory and neural networks
3. **Implement Variations**: Create your own cellular automata models
4. **Apply Insights**: Use complexity principles in your own AI projects

The journey from a simple coffee cup to understanding fundamental principles of intelligence shows how powerful simple models can be for gaining deep insights!

Keep exploring, keep questioning, and remember - complexity is everywhere! ☕🧠✨